In [1]:
!pip -q install transformers datasets


In [16]:
from datasets import load_dataset   #1 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, MarianMTModel, MarianTokenizer
import torch

imdb = load_dataset("stanfordnlp/imdb")  #2

sentimental_model = "nlptown/bert-base-multilingual-uncased-sentiment" #4
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentimental_model)  #3
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentimental_model)  #3

translation_model_name = "Helsinki-NLP/opus-mt-en-fr" #4
translation_tokenizer = MarianTokenizer.from_pretrained(translation_model_name) 
translation_model = MarianMTModel.from_pretrained(translation_model_name)  

def classify_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = sentiment_model(**inputs)

    prediction = torch.argmax(outputs.logits, dim=-1)[0].item() #5
    
    stars = "★" * (prediction + 1) #필요 시 채워주세요
    return stars
    
def translate_to_french(text):
    inputs = translation_tokenizer(text, max_length= 400, padding=True, truncation=True, return_tensors="pt") #8
    translated = translation_model.generate(**inputs)
    translated_text = translation_tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

def sentiment_and_translation_pipeline(text):
    stars = classify_sentiment(text) #9
    translated_text = translate_to_french(text) #10
        
    print(text) #6
    print(stars) #7
    print(f"Translated text: {translated_text}")
    
    return stars, translated_text


In [20]:

test_texts = [
"I absolutely loved this movie, it was a masterpiece!",
"The plot was terrible and the acting was even worse.",
"The first half of the movie was slow, but the second half really picked up and became exciting.",
"This movie was released last year and it received mixed reviews.",
"It’s the best film I’ve seen in years, truly an unforgettable experience!",
]
for text in test_texts:
    sentiment_and_translation_pipeline(text) #11



I absolutely loved this movie, it was a masterpiece!
★★★★★
Translated text: J'ai adoré ce film, c'était un chef-d'œuvre !
The plot was terrible and the acting was even worse.
★
Translated text: L'intrigue était terrible et l'acte était encore pire.
The first half of the movie was slow, but the second half really picked up and became exciting.
★★★★
Translated text: La première moitié du film a été lente, mais la deuxième moitié a vraiment repris et est devenue excitante.
This movie was released last year and it received mixed reviews.
★★★
Translated text: Ce film a été sorti l'an dernier et a reçu des critiques mitigées.
It’s the best film I’ve seen in years, truly an unforgettable experience!
★★★★★
Translated text: C'est le meilleur film que j'ai vu depuis des années, vraiment une expérience inoubliable !
